In [2]:
### PYMOO
from pymoo.core.problem import Problem, ElementwiseProblem, StarmapParallelization
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.util.display.multi import MultiObjectiveOutput
from pymoo.core.sampling import Sampling
from pymoo.core.mutation import Mutation
from pymoo.indicators.hv import HV
from pymoo.termination.max_gen import MaximumGenerationTermination
from pymoo.termination import get_termination

#multiprocessamento e outros
from tqdm.notebook import trange, tqdm
import multiprocessing, requests, sys, time, itertools, dill, random, os, pickle, copy

#Pandas, SKLearn e etc.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics, svm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold, cross_val_score

#Goatools e Gontosim
from goatools.obo_parser import GODag
from goatools.base import get_godag
#go = get_godag("go-basic.obo", optional_attrs={'relationship'})
from Similarity import Similarity_of_Two_GOTerms, Similarity_of_Set_of_GOTerms

""" Funções e Variaveis Auxiliares"""
from Constantes import *
from PlotingFunctions import *

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2024-06-17) 51,502 Terms; optional_attrs(relationship)


In [3]:
data = pd.read_pickle('Data/data_Breast_GSE70947_norm.pkl')
colunas = data.columns.drop(['type']).tolist()

affymetrix_similarity = pd.read_pickle('./Data/Breast_GSE70947-gene_symbols.pkl')
go_completness = dict(zip(affymetrix_similarity['feature'], affymetrix_similarity['scores']))
go_completness_breast_cancer = dict(zip(affymetrix_similarity['feature'], affymetrix_similarity['scores_breast_cancer_pathways']))
del go_completness['type']
del go_completness_breast_cancer['type']
X = data[colunas]
y = data['type']

In [65]:
# seleciona as features de acordo com o vetor binário
selected_features = [ 1609,  2369,  9077, 21976, 22587, 23786, 32599, 34806]
features = np.full(len(X.columns),False)
features[selected_features] = True
X_selected = np.array(X)[:,features]
f_1 = []
n_tests = 5
seed = 48
for i in range(n_tests):
    seed = seed + 1
    # Kfolding usado para separar em treino e teste
    skf = StratifiedKFold(n_splits=10, random_state=seed, shuffle=True)
    #X_train, X_valid, y_train, y_valid = train_test_split(X_selected, self.y, test_size=0.1, random_state=100)
        	
    # treino usando modelo SVM
    clf = svm.SVC(kernel='linear')
    
    # compute f_1 and AUC on validation set
    f_1.append(np.mean(cross_val_score(clf, X_selected, y, cv=skf, scoring='f1_macro')))
    print(f_1)
        
f_1 = round(np.array(f_1).sum()/n_tests, 3)
n_features = len(selected_features)
(f_1,n_features)

[0.9163295428907139]
[0.9163295428907139, 0.9203309337173351]
[0.9163295428907139, 0.9203309337173351, 0.9164847644252457]
[0.9163295428907139, 0.9203309337173351, 0.9164847644252457, 0.9132493985539035]
[0.9163295428907139, 0.9203309337173351, 0.9164847644252457, 0.9132493985539035, 0.9130957256242294]


(0.916, 8)

In [28]:
np.array(X)[:,features]

array([[-0.8408758 , -1.46618236, -0.29409045, -0.43293945, -0.72800602],
       [ 1.08865853, -2.08661464,  0.3186621 , -0.25972484, -0.71153977],
       [-0.37599821, -1.31381773,  0.1163816 ,  0.06343332, -0.81457972],
       ...,
       [-1.2523575 ,  0.46541229,  0.9711231 , -0.12686781, -0.60701292],
       [-0.19732212,  0.11519462,  0.68164375,  1.35806637,  0.57926832],
       [-0.17834283,  0.71194359, -0.55233405,  0.62100625,  0.63741638]])

In [17]:
len(X.columns)

35981